# Нейронная сеть

In [ ]:
import os
import pandas as pd
import pyarrow.parquet as pq
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import layers, models, initializers, callbacks
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Прочтите данные из файла Parquet в DataFrame Pandas
df = pd.read_csv("C:/Users/Saory-kun/Downloads/DataFrame40370.csv")

# Разделение на обучающую и тестовую выборки
X = df.drop('target', axis=1)
Y = df['target']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=88)

# Создаем архитектуру нейронной сети
model = models.Sequential([
    layers.Input(shape=(987,), name='inp'),
    layers.Dense(196, activation='sigmoid', kernel_initializer=initializers.he_normal()),
    layers.Dropout(0.2),
    layers.Dense(64, activation='sigmoid', kernel_initializer=initializers.he_normal()),
    layers.Dropout(0.2),
    layers.Dense(8, activation='sigmoid', kernel_initializer=initializers.he_normal()),
    layers.Dense(1, activation='sigmoid')
])

# Определение обратного вызова для уменьшения скорости обучения на плато
reduce_lr_callback = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Компилируем модель с использованием среднего абсолютного отклонения и оптимизатора Adam
model.compile(loss='mean_absolute_error', optimizer='adam')

# Обучение модели с использованием обратного вызова для уменьшения скорости обучения
history = model.fit(x_train, y_train, batch_size=32, epochs=500, verbose=True, validation_data=(x_test, y_test), callbacks=[reduce_lr_callback])


# Предсказание на тестовых данных
y_pred = model.predict(x_test)

# Вычисление метрик качества
precision = precision_score(y_test, y_pred.round(), average='weighted')
recall = recall_score(y_test, y_pred.round(), average='weighted')
f1 = f1_score(y_test, y_pred.round(), average='weighted')

# Вывод результатов
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Визуализация процесса обучения
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Сохранение модели
model.save("model_40370_GL_7.h5")